# Information Visualization, Altair Transforms 101
Licia He, Eytan Adar, Dallas Card

School of Information, University of Michigan

## Plan
1. Warmup - Review from 'Intro to Altair'
2. Layering 
2. Transform/Filter/Aggregate 

## Resources 
*  [Transform documentation](https://altair-viz.github.io/user_guide/transform/index.html)
*  [UW course examples](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb)
* [Reshaping documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)
* We've also created an additional lecture that shows you how to map ideas in Pandas to Altair (and vice versa)

# Altair Warmup
## Altair Transforms (part 1)

#### Import 

In [ ]:
# suppress warnings about future deprecations
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# imports we will use
import altair as alt
import pandas as pd
from vega_datasets import data as vega_data
import pprint
pp = pprint.PrettyPrinter(indent=4)

# grab the data and clean it a bit
movies_url = vega_data.movies.url
movies = pd.read_json(movies_url)
movies.columns = movies.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [ ]:
movies.sample(2)

## Warmup

Last week, we covered the basic of grammar of graphics using Altair. To create a chart, we need to specify the 

1.   Mark: e.g., point, rect, bar
2.   Data: i.e., variables and types
3.   Encoding: e.g., x, y, color 




Warm up exercise: 

 ![1.1](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-1.1.png)

In [ ]:
#1.1 Exercise: basic bar chart for Major_Genre and avg of Production Budget 
alt.Chart(movies).mark.encode(
  
)

We can stitch multiple charts together by making a [compound chart](https://altair-viz.github.io/user_guide/compound_charts.html). Specifically, we discussed **horizontal concatenation** (hconcat or |) and **vertical concatenation** (vconcat or & )

Warmup #2 

 ![1.2](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-1.2.png)

In [ ]:
#1.2 Exercise 2: average vs median 
# basic bar chart for Major_Genre and avg of Production Budget 
# basic bar chart for Major_Genre and median of Production Budget 
# put them side by side 

chart1=alt.Chart(movies).mark.encode(
   
)

chart2=alt.Chart(movies).mark.encode(
    
)
# chart1&chart2



You can also build more sophisticated visualizations by combining charts both horizontally and vertically. For example, this next plot is known as a scatterplot with marginal historgrams. In the middle we are looking at the correlation between Rotten Tomatoes and IMDB scores. On the top and right we see the distributions of each variable separately. In this case we see a weak correlation and also note the IMDB rating has a nice normal distribution but the Rotten tomatoes scores seem more uniform. 

 ![1.3](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-1.3.png)

In [ ]:
# 1.3 

# scatter plot for the middle
points = alt.Chart(movies).mark_.encode(
    
)

points

In [ ]:
# the histogram to put on the right side (notice we remove the axes)
distribright= alt.Chart(movies).mark.encode(
    
).properties(width=30)

# the histogram to put on the top (notice we remove the axes)
distribtop = alt.Chart(movies).mark.encode(
    
).properties(height=30)


In [ ]:
# put the scatter and right together side by side and then put those
# under the top histogram


In [ ]:
# 1.4 (really basic interactivity)
# lots of movies on top of each other

points.interactive()

In [ ]:
# 1.5 
# How do we know what the movies are?
# Tooltip channel - add information as a tooltip



In [ ]:
# 1.6 
# Extended tooltips: Use tooltips as an array



# Altair Layering


## Plan
1. Warmup
2. Layering (you are here)
2. Transform/Filter/Aggregate 

[Layering](https://altair-viz.github.io/user_guide/compound_charts.html#layered-charts) is a very useful compounding method that allows you to overlay two different charts on the same set of axes. You can layer charts using the "+" operator. 

### Our Goal:

![1.4](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-1.4.png)

In [ ]:
#1.4 copy 1.2, change median to a line 
chart1=
chart2=




In [ ]:
# 1.4 Shortcut
chart1=alt.Chart(movies).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X('average(Production_Budget):Q')
)

# take the first chart and *override* the values
# bar --> line
# average on X --> median
# everything else stays the same (data, Y, etc.)
chart2=

chart1+chart2

In [ ]:
# 1.4 (one last variant)

# the "base chart" (not for rendering)
base = alt.Chart(movies).encode(
    alt.Y('Major_Genre:N')
)

# overriden versions
chart1=
chart2=

chart1+chart2

Layering is also commonly used to add text annotations. 

### Our goal (ugly version):
![1.5](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-1.5.png)

In [ ]:
#1.5 layering text 

# our "base chart"
chart1=alt.Chart(movies).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X('average(Production_Budget):Q')
)

# Our text chart "overrides" elements of the base
# in this case we are replacing the mark type and
# adding an encoding
text_annotation_for_mean = 


chart1+text_annotation_for_mean

In [ ]:
## 1.5 Make it look a little better

#adopt settings from the previous chart
text_annotation_for_mean = chart1.mark_text(
    align='left',
    dx=7, #horizontal distance between end of bar to the start of text 
).encode(
    alt.Text(
        'mean(Production_Budget)', 
        formatType="number", #necessary for formatting numbers
        format="$s" #formatting numbers into millions. 
    ),
)

chart1+text_annotation_for_mean

# Altair Transforms
## Altair Transforms (part 3)
Licia He, Eytan Adar, Sereen Kallerackal, and Dallas Card


School of Information, University of Michigan

## Plan
1. Warmup
2. Layering 
2. Transform/Filter/Aggregate (you are here) 

### Why Do We Transform?

Often data doesn't "look" the way we need it for visualization
* too much data
* not enough data
* data in the wrong shape
* data isn't aggregated
* data is missing
* etc.

Altair provides 13 different types of transformations. These will allow you to perform basic manipulation of the data without having to use some external tool like Pandas. Although Pandas may be more powerful, the advantage of doing transformations directly in Altair/Vega-Lite is you will be able to deploy your code without needing Python (just Vega-Lite).

## Pandas vs Altair

* You can usually achieve the transform both ways 
* With Pandas you'll be stuck running a Python server (can't deploy)
 * "Fix" is to transform in Pandas and save the intermediate result
 * But problematic for dynamic data
* With Altair, it'll be harder to debug


| Transform                | Description                                                                     |
|:-------------------------|:--------------------------------------------------------------------------------|
| Aggregate Transforms     | Create a new data column by aggregating an existing column.                     |
| Bin transforms           | Create a new data column by binning an existing column.                         |
| Calculate Transform      | Create a new data column using an arithmetic calculation on an existing column. |
| Filter Transform         | Select a subset of data based on a condition.                                   |
| Flatten Transform        | Flatten array data into columns.                                                |
| Fold Transform           | Convert wide-form data into long-form data.                                     |
| Impute Transform         | Impute missing data.                                                            |
| Join Aggregate Transform | Aggregate transform joined to original data.                                    |
| Lookup Transform         | One-sided join of two datasets based on a lookup key.                           |
| Sample Transform         | Random sub-sample of the rows in the dataset.                                   |
| Stack Transform          | Compute stacked version of values.                                              |
| TimeUnit Transform       | Discretize/group a date by a time unit (day, month, year, etc.)                 |
| Window Transform         | Compute a windowed aggregation                                                  |

### 2.1 Aggregate and join aggregate  

Last class we covered [aggregate transformation](https://altair-viz.github.io/user_guide/transform/aggregate.html). For example, using mean, max, min, sum to compute aggregate summary statistics over groups of data.

In [ ]:
#2.1.1 copy of 1.1 
# calculatge the average production budget per genre

alt.Chart(movies).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X('average(Production_Budget):Q'),
)

You can also use the longer form to plot the same chart


In [ ]:
#2.1.2 copy 2.1.1 and adjust to longer form 
# this is the explicit form of the code above
alt.Chart(movies).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X(field='Production_Budget', aggregate="mean", type="quantitative") 
    #can't use Q here
)

The same chart could be made with an explicit computted aggregation. 


In [ ]:
#2.1.3 copy 2.1.2 and change to transform_aggregate 
# With transform aggregate, you need to specific what you want to group_by with.
# You can group_by multiple variables

alt.Chart(movies).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X(field='Production_Budget', aggregate="mean", type="quantitative") 
)

If you create an aggregated variable and didn't use it, what will happen? 

In [ ]:
#2.1.4 , copy 2.1.3 aggregate without usg 
# original data is impacted, the mean_production_budget is not available for other 
# transformation, such as filter 
# This happens because the data is replaced with mean_production_budget, and the other og data doesn't exist. So that is why nothing comes up.

alt.Chart(movies).transform_aggregate(
    groupby=['Major_Genre'],
    mean_production_budget='mean(Production_Budget)'
).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X('mean(Production_Budget):Q')                         # Production_Budget goes away
)

#### Aggregate vs. JoinAggregate
Let's take a look at this mini example 


| Title | Genre | Production_Budget |
|-------|-------|-------|
| A     | x     | 1     |
| B     | x     | 2     |
| C     | y     | 10    |

transform_aggregate will **change the original data structure** and group rows together. 


e.g. running the following transformation 
``` 
.transform_aggregate(
    groupby=['Major_Genre'],
    mean_production_budget='mean(Production_Budget)'
) 
```
will create a table like this 

| Major_Genre | mean_production_budget |
|-------|------------------------|
| x     | 1.5                    |
| y     | 10                     |

If we want to **preserve the original data structure**, we will use joinAggregate. This creates two datasets - the original one and the new one with the new variable

e.g. running the following transformation 
``` 
.transform_joinaggregate(
    groupby=['Major_Genre'],
    mean_production_budget='mean(Production_Budget)'
) 
```

will generate the following table 

| Title | Major_Genre | Value | mean_production_budget |
|-------|-------|-------|------------------------|
| A     | x     | 1     | 1.5                    |
| B     | x     | 2     | 1.5                    |
| C     | y     | 10    | 10                     |

See the [join aggregate transform](https://altair-viz.github.io/user_guide/transform/joinaggregate.html) for more information.

In [ ]:
#2.1.5, copy 2.1.4 but use transform_joinaggregate instead
alt.Chart(movies).transform_aggregate(
    groupby=['Major_Genre'],
    mean_production_budget='mean(Production_Budget)'
).mark_bar().encode(
    alt.Y('Major_Genre:N'),
    alt.X('mean(Production_Budget):Q')                         # Production_Budget still there
                                                        #add color encoding
)


### 2.2 Bin Transformation 

We discussed the [bin transformation](https://altair-viz.github.io/user_guide/transform/bin.html#user-guide-bin-transform), replicate the following visualization: 

![2.2.1](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-2.2.1.png)

In [ ]:
#2.2.1, EXERCISE barchart with binned IMDB_rating and mean Production_Budget
alt.Chart(movies).mark.encode(
    
)

In [ ]:
# 2.2.2 the "long form" way
alt.Chart(movies).mark_bar().transform_bin(
    
).encode(
   
)

What happens if we are missing data? Or need to create/calculate new fields?

### 2.3 Calculate Transform 
The [calculate transform ](https://altair-viz.github.io/user_guide/transform.html#calculate-transform)allows the user to define new fields in the dataset which are calculated from other fields using an expression syntax.

For example, we want to have a column called "**Revenue**" that's equal to the difference between Worldwide_Gross and Production_Budget

In [ ]:
## 2.3.1 Revenue vs Major_Genre
alt.Chart(movies).mark_point().encode(
    alt.X('Major_Genre:N'),
    alt.Y('Revenue:Q')
)

In [ ]:
## 2.3.2 Revenue vs Major_Genre (Alternative way)
alt.Chart(movies).transform_calculate(
    Revenue= #can use datum instead of alt.datum
).mark_point().encode(
    alt.X('Major_Genre:N'),
    alt.Y('Revenue:Q')
)

You can chain multiple transformations together. Make a bar chart for mean revenue and major_genre

![2.3.2](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-2.3.2.png)

In [ ]:
## 2.3.2 


In [ ]:
#2.3.4 Color movies by whether they are cheap or expensive 
# Excercise: We want to encode production budget using a nominal variable(expensive/cheap) ~ so we have to create a condition

# uses the javascript shorthand to write the condition
    # condition ? expression_1 (if true) : expression_2 (if false)
    # if production budget>200000000 then 'expensive', else 'cheap'
alt.Chart(movies).transform_calculate(
    expensiveOrCheap=
).mark_point().encode(
    alt.X('Major_Genre:N'),
    alt.Y('Production_Budget:Q')
)


There will be situations when we have too much data. In this case we will use Filter Transform.

### 2.4 Filter Transform 
The[ filter transform ](https://altair-viz.github.io/user_guide/transform/filter.html#user-guide-filter-transform)removes objects from a data stream based on a provided filter expression, selection, or other filter predicate. 

There are multiple ways of specifying filters. The first way is using a Vega expression. 

In [ ]:
#2.4.1 alt.datum: & for and, | of or, > < = != 

# What's the mean production budget for movies with more than 
# 500 votes and a rating > 5

# & for and, | of or, > < = != 
alt.Chart(movies).transform_filter(
    
).mark_bar().encode(
    alt.X('IMDB_Rating:Q'),
    alt.Y('mean(Production_Budget)')
)

In [ ]:
#2.4.2

alt.Chart(movies).transform_filter(
    # & for and, | of or, > < = != 
    # javascript way
   
).mark_bar().encode(
    alt.X('IMDB_Rating:Q'),
    alt.Y('mean(Production_Budget)')
)

You can also make the same chart using **Field Predicates**. Field predicates overlap somewhat in function with expression predicates, but have the advantage that their contents are validated by the schema. Examples are:

In [ ]:
#Filters are basically done by using Field Predicates
#2.4.3 using predicates: equal, lt(less than), gt, lte, gte(greater than or equal to) 

alt.Chart(movies).transform_filter(
    #equal, lt, gt, lte, gte 
    

    
).mark_bar().encode(
    alt.X('IMDB_Rating:Q'),
    alt.Y('mean(Production_Budget)')
)

Here are 2 very useful field predicates: 
* **FieldOneOfPredicate** evaluates whether a field is among a list of specified values.
* **FieldRangePredicate** evaluates whether a continuous field is within a range of values.

In [ ]:
# 2.4.3 oneOf and Range:  
# include only Drama and Comedy movies that have ratings between 5 and 7. 
alt.Chart(movies).transform_filter(
    #equal, lt, gt, lte, gte 
    alt.FieldOneOfPredicate(field='Major_Genre', oneOf=['Drama','Comedy'])
).transform_filter(
    alt.FieldRangePredicate(field='IMDB_Rating', range=[5,8])
).mark_bar().encode(
    alt.X('IMDB_Rating:Q'),
    alt.Y('mean(Production_Budget)')
)

instead of !, &  and |, you can also use [logical operands](https://altair-viz.github.io/user_guide/transform/filter.html#logical-operands)

In [ ]:
#2.4.4 copy 2.4.3, use logical operands instead of using transform_filter multiple times
#feeding in a Json object, logical operator we want to use is 'and' and we are applying 'and' operator on the lists of variables
#not-recommended option
alt.Chart(movies).transform_filter(
    #equal, lt, gt, lte, gte 
    alt.FieldOneOfPredicate(field='Major_Genre', oneOf=['Drama','Comedy'])
).transform_filter(
    alt.FieldRangePredicate(field='IMDB_Rating', range=[5,8])
).mark_bar().encode(
    alt.X('IMDB_Rating:Q'),
    alt.Y('mean(Production_Budget)')
)


### 2.4.5 Sample Transform 
[Sample Transform ](https://altair-viz.github.io/user_guide/transform/sample.html)lets you specify a number of rows to randomly choose from the dataset. 

In [ ]:
#2.4.5 SAMPLE, run the following code  
alt.Chart(movies).mark_bar().encode(
    alt.X('IMDB_Rating:Q'),
    alt.Y('Title:N')
).transform_sample(10) #run it again and see what happens ~ keeps changing

So far we have been using tranformations for one row/a single row. Window transformations can be used for takining into consideration multiple rows at the same time.

### 2.4.6 Window Transformation 
[window transformation](https://altair-viz.github.io/user_guide/transform/window.html) calculates over sorted groups of data objects. These calculations include ranking, lead/lag analysis, and aggregates such as cumulative sums and averages. 

#### Produce the following:

![2.4.7](https://raw.githubusercontent.com/eytanadar/si649public/master/lab4/assets/demo/altair-2.4.7.png)

In [ ]:
#2.4.6: Ranking through the Window Transform

alt.Chart(movies,width=600)